# Предварительная обработка данных
Приведение всех признаков к числовым и удаление ненужных

## Описание признаков

| Признак        | Описание                                                                                                                                         |
|----------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| **PassengerId** | Уникальный числовой идентификатор пассажира. Используется для однозначной идентификации каждого пассажира в наборе данных.                        |
| **Survived**    | Флаг, указывающий, выжил ли пассажир. Значения: <br> - `0` = Нет (погиб), <br> - `1` = Да (выжил).                                               |
| **Pclass**      | Класс билета пассажира, который также отражает социально-экономический статус. Значения: <br> - `1` = Первый класс, <br> - `2` = Второй класс, <br> - `3` = Третий класс. |
| **Name**        | Полное имя пассажира, включая титул (например, Mr., Mrs., Miss и т.д.).                                                                          |
| **Sex**         | Пол пассажира. Возможные значения: <br> - `male` = Мужчина, <br> - `female` = Женщина.                                                           |
| **Age**         | Возраст пассажира в годах. Может содержать пропущенные значения (NaN).                                                                           |
| **SibSp**       | Количество братьев/сестер или супругов на борту Титаника.                                                                                        |
| **Parch**       | Количество родителей или детей на борту Титаника.                                                                                                |
| **Ticket**      | Номер билета пассажира. Может быть строковым или числовым значением.                                                                             |
| **Fare**        | Стоимость билета пассажира. Значение может быть дробным числом.                                                                                  |
| **Cabin**       | Номер каюты пассажира. Может содержать пропущенные значения (NaN).                                                                               |
| **Embarked**    | Порт посадки пассажира. Возможные значения: <br> - `C` = Cherbourg, <br> - `Q` = Queenstown, <br> - `S` = Southampton.                           |

## Подключение библиотек

In [11]:
import pandas as pd

## Обработка

In [12]:
datasetType = 'train' 
dfOrignal = pd.read_csv('{}.csv'.format(datasetType))
df = dfOrignal.copy()
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


### Удаляем данные, которые никак не повлияют на таргетное значение

In [13]:
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


### Вычисление процентного соотношения пропущенных и непропущенных значений

In [14]:
missingPercent = df.isnull().mean() * 100
nonMisssingPercent = (1 - df.isnull().mean()) * 100

missingStats = pd.DataFrame({
    "Total": df.isnull().sum(), 
    "Percent Missing": missingPercent.round(2),  
    "Percent Non-Missing": nonMisssingPercent.round(2) 
})

print(missingStats)

          Total  Percent Missing  Percent Non-Missing
Survived      0             0.00               100.00
Pclass        0             0.00               100.00
Sex           0             0.00               100.00
Age         177            19.87                80.13
SibSp         0             0.00               100.00
Parch         0             0.00               100.00
Ticket        0             0.00               100.00
Fare          0             0.00               100.00
Cabin       687            77.10                22.90
Embarked      2             0.22                99.78


### Удаление столбцов с более чем 50% пропущенных значений

In [15]:
columnsToDrop = missingPercent[missingPercent > 50].index
df.drop(columnsToDrop, axis=1, inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S


### Преобразование категориальных признаков в вещественные значения


In [16]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,0,22.0,1,0,A/5 21171,7.2500,2.0
1,1,1,1,38.0,1,0,PC 17599,71.2833,0.0
2,1,3,1,26.0,0,0,STON/O2. 3101282,7.9250,2.0


### Удаляем признак Ticket, так как непонятно по какому правилу преобразовать его в число

In [17]:
df.drop(['Ticket'], axis=1, inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2.0
1,1,1,1,38.0,1,0,71.2833,0.0
2,1,3,1,26.0,0,0,7.9250,2.0


### Заполнение пропущенных значений в признаках Age и Embarked

In [18]:
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2.0
1,1,1,1,38.0,1,0,71.2833,0.0
2,1,3,1,26.0,0,0,7.9250,2.0
3,1,1,1,35.0,1,0,53.1000,2.0
4,0,3,0,35.0,0,0,8.0500,2.0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,2.0
887,1,1,1,19.0,0,0,30.0000,2.0
888,0,3,1,21.5,1,2,23.4500,2.0
889,1,1,0,26.0,0,0,30.0000,0.0


### Сохранение обработанного датасета

In [20]:
df.to_csv('preprocessed_{}.csv'.format(datasetType), index=False)